# TAREA 1 B

# 1. Selección de Dataset

El dataset seleccionado se trata de Jena Climate Dataset: https://www.kaggle.com/datasets/mnassrib/jena-climate

Este dataset contiene la información de 420551 fechas, para cada fecha se estudian una serie de características climatológicas en la Estación meteorológica del Instituto Max Planck de Biogeoquímica en Jena (Alemania).

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/jena_climate.csv'
data = pd.read_csv(file_path)

# 2. Análisis Preliminar

Para cada instante de tiempo se estudian las siguientes características:

*   Date Time: fecha con la información del año, mes, dia, hora, minuto y segundo
*   p (mbar): presión
*   T (degC): temperatura
*   Tpot (K): temperatura potencial
*   Tdew (degC): temperatura de punto de condensación
*   rh (%): humedad relativa
*   VPmax (mbar): presión máxima de vapor
*   VPact (mbar): presión de vapor real
*   VPdef (mbar): déficit de presión de vapor
*   sh (g/kg): humedad específica
*   H2OC (mmol/mol): concentración de vapor de agua
*   rho (g/m**3): densidad del aire
*   wv (m/s): velocidad del viento
*   max. wv (m/s): velocidad máxima del viento
*   wd (deg): dirección del viento

El caso de estudio al que nos enfrentamos es el de una empresa de agricultura que trabaja con cultivos de invernadero, por lo que el control de la temperatura en sus plantaciones requiere de un cuidado extremo.

Dada esta situación, se nos pide desarrollar un modelo de inteligencia artificial que, en base a diversos parámetros meteorológicos, pueda predecir la temperatura que habrá en una fecha a fin de que la empresa pueda ajustar la temperatura interna del invernadero para ese momento.

In [ ]:
# Displaying the DataFrame structure
print("DataFrame Structure:")
print(data.head())

# 1. Data types of each column
print("1. Data Types:")
print(data.dtypes)

# 2. Column labels
print("\n2. Columns:")
print(data.columns)

# 3. Dimensions of the DataFrame
print("\n3. Shape:")
print(data.shape)

# 4. Index (row labels) of the DataFrame
print("\n4. Index:")
print(data.index)

# 5. Number of elements in the DataFrame
print("\n5. Size:")
print(data.size)

# 6. Basic information about DataFrame structure
print("\n6. Basic Information about DataFrame:")
print(data.info())

# 7. Summary statistics for numerical columns
print("\n7. Summary Statistics for Numerical Columns:")
print(data.describe())

# 8. Missing values
print("\n9. Checking for Missing Values:")
print(data.isnull().sum())

DataFrame Structure:
             Date Time  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  rh (%)  \
0  01.01.2009 00:10:00    996.52     -8.02    265.40        -8.90    93.3   
1  01.01.2009 00:20:00    996.57     -8.41    265.01        -9.28    93.4   
2  01.01.2009 00:30:00    996.53     -8.51    264.91        -9.31    93.9   
3  01.01.2009 00:40:00    996.51     -8.31    265.12        -9.07    94.2   
4  01.01.2009 00:50:00    996.51     -8.27    265.15        -9.04    94.1   

   VPmax (mbar)  VPact (mbar)  VPdef (mbar)  sh (g/kg)  H2OC (mmol/mol)  \
0          3.33          3.11          0.22       1.94             3.12   
1          3.23          3.02          0.21       1.89             3.03   
2          3.21          3.01          0.20       1.88             3.02   
3          3.26          3.07          0.19       1.92             3.08   
4          3.27          3.08          0.19       1.92             3.09   

   rho (g/m**3)  wv (m/s)  max. wv (m/s)  wd (deg)  
0       1307

# 3. Preparación del Dataset

En este caso de estudio no existen variables categóricas y no encontramos valores nulos que imputar.

Solo sería necesario transformar la columna "Date Time" a un tipo más apropiado como datetime64 en lugar de float64.

In [ ]:
# Convert Date Time column type from float64 to datetime64

data['Date Time'] = pd.to_datetime(data['Date Time'], format = "%d.%m.%Y %H:%M:%S")

# Separate features and target variable

x = data.drop(columns = ['T (degC)'], axis=1)

y = data['T (degC)']

# 4. Implementación de Cross-Validation
Mediante técnicas de cross-validation realizaremos 5 evaluaciones de la capacidad de predicción que tiene nuestro modelo, para este propósito he decidido utilizar TimeSeriesSplit como técnica de cross-validation, esta técnica está específicamente diseñada para datasets con series temporales debido a que respeta la estructura temporal de los datos.

Para ello al dividir los conjuntos asegura que el conjunto de test siempre se encuentre después de los conjuntos de entrenamiento, ya que los datos que el modelo pretende predecir siempre serán posteriores respecto a aquellos para los que el modelo ya cuenta con información previa. Evitando de este modo la aparición de data leakage al impedir que se entrene con información del futuro.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Create instance of TimeSeriesSplit technique for 5 subsets
tscv = TimeSeriesSplit(n_splits=5)

# 5. Entrenamiento del Modelo

Respecto a la elección del algoritmo para entrenar el modelo, teniendo en cuenta que se trata de un problema de regresión, he decidido apostar por utilizar LinearRegression debido a que suele darse una relación lineal entre variables climatológicas, ej: a mayor frío se da mayor cantidad de precipitaciones.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline

for train, test in tscv.split(x, y):

  x_train, x_test = x.iloc[train], x.iloc[test]
  y_train, y_test = y.iloc[train], y.iloc[test]

  # Identify numerical columns
  numerical_cols = x_train.select_dtypes(include=['float64']).columns

  # Create transformer for numerical features
  numerical_transformer = StandardScaler()

  # Configure preprocessor to transform numerical variables
  preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols)])

  # Create pipeline to transform the data and train the model
  model = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())])

  # Apply the model to the training data
  model.fit(x_train, y_train)

  # Evaluate the model with the testing data
  y_pred = model.predict(x_test)

  print("Model Evaluation:")
  mae = mean_absolute_error(y_test, y_pred)
  print(f'MAE: {mae}\n')

Model Evaluation:
MAE: 0.00834462345115233

Model Evaluation:
MAE: 0.007049643768562205

Model Evaluation:
MAE: 0.007026506075041364

Model Evaluation:
MAE: 0.006746114616356486

Model Evaluation:
MAE: 0.008316091267780783



# 6. Evaluación y Discusión:

Para la evaluación de métricas del modelo optaré por utilizar mean_absolute_error, que permite cuantificar el error promedio absoluto entre las predicciones y los valores reales.

Podemos observar que a lo largo de todas las iteraciones se obtiene un error promedio absoluto extremadamente bajo, lo cual es garantía de que el modelo realiza predicciones precisas, es robusto y generaliza correctamente a datos no vistos.

La implementación de cross-validation nos aporta una mayor confianza a la hora de emitir nuestros resultados, pues partiendo de un mismo dataset podemos obtener tantas evaluaciones como deseemos de nuestro modelo, ayudándonos a observar como las evaluaciones muestran una tendencia similar entre las distintas iteraciones.

Debido a que el caso que nos ocupa es un problema de series temporales, si eligiésemos incorrectamente la técnica de cross-validation podríamos tener un problema de data leakage, puesto que podríamos entrenar con datos de fechas futuras el modelo e intentar predecir casos de fechas pasadas.